In [1]:
import requests
import random
import time
import json
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive
import os


In [4]:
# Mount Google Drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/StockData/'

# Create folder if it doesn't exist
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

def get_stock_data(ticker):
    url = f'https://finance.yahoo.com/quote/{ticker}'
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; win64; x64) Applewebkit/S37.36 (KHTML, Like Gecko) Chrome/127.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Applevebkit/537,36 (KHTML, Like Gecko) Chrome/125.e.e.e safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) Applewebkit/537.36 (KHTML, like Gecko) chrome/124.0.8.0 Safari/537,36'
    ]

    max_retries = 10

    for attempt in range(max_retries):
        headers = {'User-Agent': random.choice(user_agents)}

        try:
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                price_element = soup.find('span', {'data-testid': 'qsp-price'})
                price_change_element = soup.find('span', {'data-testid': 'qsp-price-change'})
                price_change_percent_element = soup.find('span', {'data-testid': 'qsp-price-change-percent'})

                if not (price_element and price_change_element and price_change_percent_element):
                    return {}

                price = price_element.text
                price_change = price_change_element.text
                price_change_percent = price_change_percent_element.text

                stats_labels = [label.text for label in soup.select('div[data-testid="quote-statistics"] ul li span:first-child')]
                stats_values = [value.text for value in soup.select('div[data-testid="quote-statistics"] ul li span:last-child')]

                financial_data = dict(zip(stats_labels, stats_values))

                data = {
                    'Stock': ticker,
                    'Price': price,
                    'Change': price_change,
                    'Change (%)': price_change_percent,
                    **financial_data
                }

                return data
            elif response.status_code == 429:
                time.sleep(10 * (attempt + 1))

        except Exception as e:
            print(f"Error occurred: {e}")
            pass

    return {}

def main():
    ticker_symbols = []
    while True:
        ticker = input("Enter a stock ticker (or type 'Stop' to finish): ").strip()
        if ticker.lower() == 'stop':
            break
        ticker_symbols.append(ticker.upper())

    stockdata = [get_stock_data(symbol) for symbol in ticker_symbols if get_stock_data(symbol)]

    if stockdata:
        df = pd.DataFrame(stockdata)

        # Save the JSON file to Google Drive
        with open(os.path.join(drive_path, 'stock_profile_data.json'), 'w') as json_file:
            json.dump(stockdata, json_file, indent=4)

        df.to_csv(os.path.join(drive_path, 'stock_profile_data.csv'), index=False)
        df.to_excel(os.path.join(drive_path, 'stock_profile_data.xlsx'), index=False)

        print("Data has been saved successfully to Google Drive.")

main()


Mounted at /content/drive
Enter a stock ticker (or type 'Stop' to finish): TSLA
Enter a stock ticker (or type 'Stop' to finish): MSFT
Enter a stock ticker (or type 'Stop' to finish): COST
Enter a stock ticker (or type 'Stop' to finish): PYPL
Enter a stock ticker (or type 'Stop' to finish): ADBE
Enter a stock ticker (or type 'Stop' to finish): PEP
Enter a stock ticker (or type 'Stop' to finish): CVX
Enter a stock ticker (or type 'Stop' to finish): GOOGL
Enter a stock ticker (or type 'Stop' to finish): STOP
Data has been saved successfully to Google Drive.
